In [ ]:
""" MODELO PRE ENTRENADO VGG - FINE TUNING PCAM
Dataset: Patch Camelyon, desde tensorflow datasets
Fuente del modelo: keras_hub
Modelo: vgg_19_imagenet
"""

' MODELO PRE ENTRENADO VGG - FINE TUNING PCAM\nDataset: Patch Camelyon, desde tensorflow datasets\nFuente del modelo: keras_hub\nModelo: vgg_19_imagenet\n'

In [ ]:
# Conectar con Google Drive
from google.colab import drive
_ = drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM"


Mounted at /content/drive


In [ ]:
!pip install tensorflow --upgrade
!pip install keras_cv
!pip install keras_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow

In [ ]:
import tensorflow as tf
import keras
import keras_hub
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Leer datasets PCam desde copia en disco:
ruta_pcam_tr = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_train"
train_ds = tf.data.Dataset.load(ruta_pcam_tr)
ruta_pcam_val = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_val"
val_ds = tf.data.Dataset.load(ruta_pcam_val)

# Usar la función 'map' para dejar solamente la imagen y la etiqueta
# quitando la identificación de la imagen
train_ds = train_ds.map(lambda x: (x["image"], x["label"]), \
                        num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x: (x["image"], x["label"]), \
                        num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
print("len(train_ds):",len(train_ds))
print("len(val_ds):",len(val_ds))

len(train_ds): 262144
len(val_ds): 32768


In [ ]:
#  Instancier el preprocesador de entrada para normalizar las imagenes al rango 0..1
preprocessor = keras.layers.Pipeline([keras.layers.Rescaling(1.0 / 255)])
# Preparar el dataset en lotes y usar prefetch para optimizar el entrenamiento
batch_size = 32
train_ds = train_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Instanciar el clasificador de imagenes basado en resnet
# model = keras_hub.models.VGGBackbone.from_preset("vgg_19_imagenet")
image_classifier = keras_hub.models.ImageClassifier.from_preset(
    "vgg_19_imagenet",
    activation="softmax",
    num_classes=2,
    preprocessor=preprocessor,
)
# Compilar el modelo:
image_classifier.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

100%|██████████| 495/495 [00:00<00:00, 799kB/s]


100%|██████████| 3.03k/3.03k [00:00<00:00, 3.78MB/s]


100%|██████████| 548M/548M [00:11<00:00, 49.8MB/s]


100%|██████████| 76.4M/76.4M [00:01<00:00, 40.9MB/s]


In [ ]:
# Entrenar 3 epocas
%%time
image_classifier.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
  )

Epoch 1/3
8192/8192 ━━━━━━━━━━━━━━━━━━━━ 1921s 231ms/step - accuracy: 0.5001 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931
Epoch 2/3
8192/8192 ━━━━━━━━━━━━━━━━━━━━ 1897s 228ms/step - accuracy: 0.5003 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931
Epoch 3/3
8192/8192 ━━━━━━━━━━━━━━━━━━━━ 1899s 230ms/step - accuracy: 0.5003 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931
CPU times: user 11min 12s, sys: 4min 30s, total: 15min 43s
Wall time: 1h 36min 12s


In [ ]:
# Cargar el dataset de prueba
ruta_pcam_test = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_test"
test_ds = tf.data.Dataset.load(ruta_pcam_test)
# Usar la función 'map' para dejar solamente la imagen y la etiqueta
# quitando la identificación de la imagen
test_ds = test_ds.map(lambda x: (x["image"], x["label"]), \
                        num_parallel_calls=tf.data.AUTOTUNE)
print("len(test_ds):",len(test_ds))
# Preparar el dataset en lotes y usar prefetch para optimizar la evaluación
test_ds = test_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)


len(test_ds): 32768


In [ ]:
%%time
_, accuracy = image_classifier.evaluate(test_ds)
print(f"Accuracy en el dataset de prueba: {accuracy}")

1024/1024 ━━━━━━━━━━━━━━━━━━━━ 65s 63ms/step - accuracy: 0.4971 - loss: 0.6932
Accuracy en el dataset de prueba: 0.499786376953125
CPU times: user 16.3 s, sys: 3.63 s, total: 19.9 s
Wall time: 1min 21s
